In [2]:
from pathlib import Path
#from itertools import combinations
import re
import numpy as np
import pandas as pd
#from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import NegativeClassOptimization.config as config
import yaml
import os
import glob

### Saving test datasets with true and shuffeled labels
Bellow I retrive test datasets chosen by immuneML, they are different for each pair even though one antigen matches

In [182]:
create_suffled = False
if create_suffled:
    path_suffix = "pairwise_wo_dupl_rf_out"
    regex_file_name = "fit_pairwise_dataset_*_instruction"
    BASE_PATH = config.IMMUNE_ML_BASE_PATH/ f"{path_suffix}"
    paths = list(Path(BASE_PATH).glob(regex_file_name))
    for path in paths:
        ag1, ag2 = antigens_from_dir_name(str(path))
        test_path = list((path / 'split_1/datasets').glob('*_test_*.npy'))[0]
        df_test= pd.DataFrame(np.load(test_path))
        train_test_path = config.DATA_BASE_PATH/ f'pairwise_wo_dupl/pairwise_dataset_{ag1}_{ag2}.tsv'
        df_train_test = pd.read_csv(train_test_path, sep='\t')
        df_test = pd.merge(df_train_test, df_test, left_on='Slide', right_on = 'amino_acid_sequence')[['UID', 'Slide', 'Antigen','binder']]
        df_test.to_csv(config.DATA_BASE_PATH / f'test_datasets/original/test_dataset_{ag1}_{ag2}.tsv', sep='\t')
        random_len = df_test.shape[0]
        random_labels = np.random.randint(2,size=random_len)
        df_test['binder'] = (random_labels==1)
        df_test.to_csv(config.DATA_BASE_PATH / f'test_datasets/shuffled/test_dataset_{ag1}_{ag2}.tsv', sep='\t')
        print(f'Saved test dataset for {ag1} {ag2}')

Saved test dataset for 2YPV 3VRL
Saved test dataset for 1H0D 3VRL
Saved test dataset for 1ADQ 2YPV
Saved test dataset for 3RAJ 5E94
Saved test dataset for 1FBI 1H0D
Saved test dataset for 1H0D 1OB1
Saved test dataset for 1ADQ 1NSN
Saved test dataset for 3VRL 5E94
Saved test dataset for 1OB1 5E94
Saved test dataset for 1H0D 1WEJ
Saved test dataset for 1WEJ 5E94
Saved test dataset for 1NSN 3RAJ
Saved test dataset for 1ADQ 1FBI
Saved test dataset for 1FBI 5E94
Saved test dataset for 1NSN 1WEJ
Saved test dataset for 1ADQ 5E94
Saved test dataset for 1H0D 3RAJ
Saved test dataset for 2YPV 3RAJ
Saved test dataset for 1FBI 2YPV
Saved test dataset for 1NSN 3VRL
Saved test dataset for 1FBI 1NSN
Saved test dataset for 1WEJ 2YPV
Saved test dataset for 1ADQ 1H0D
Saved test dataset for 1NSN 1OB1
Saved test dataset for 1OB1 2YPV
Saved test dataset for 1ADQ 3VRL
Saved test dataset for 1ADQ 1OB1
Saved test dataset for 1H0D 1NSN
Saved test dataset for 1H0D 2YPV
Saved test dataset for 1FBI 3RAJ
Saved test

In [117]:
def pairwise_test_yaml(datasets_path, regex_datasets_name, models_root, spec_name):
    datasets = dict()
    instructions = dict()
    paths = list(Path(datasets_path).glob(regex_datasets_name)) #config.DATA_BASE_PATH / 'test_datasets'
    for path in paths:
        path = str(path)
        ag1, ag2 = re.search('test_dataset_(.*)_(.*).tsv', path).groups()
        config_path = config.IMMUNE_ML_BASE_PATH / f'{models_root}/fit_pairwise_dataset_{ag1}_{ag2}_instruction/optimal_binder/zip/ml_settings_binder.zip'
        config_path = str(config_path)
        datasets[f'{spec_name}_{ag1}_{ag2}'] = {
                "format": "Generic",
                "params": {
                    "path": path,
                    "is_repertoire": False,
                    "region_type": "FULL_SEQUENCE",
                     "column_mapping": {
                            "Slide": "sequence_aas",
                             "UID": "sequence_identifiers"

                        },
                        "metadata_column_mapping": {
                            "Antigen": "Antigen",
                            "binder": "binder",
                        }}
        }
        instructions[f'{spec_name}_{ag1}_{ag2}_instruction'] = {
                    "type": "MLApplication",
                    "dataset": f'{spec_name}_{ag1}_{ag2}',
                    "config_path": config_path,
                    "number_of_processes": 4
        }
    return datasets, instructions

In [118]:
#BASE_PATH = config.IMMUNE_ML_BASE_PATH/ 'pairwise_wo_dupl_rf_out'
#paths = list(Path(BASE_PATH).glob("fit_pairwise_dataset_*_instruction"))

In [125]:
class NoAliasDumper(yaml.SafeDumper):
    """
    Yaml dumper without yaml references.
    """
    def ignore_aliases(self, data):
        return True

RF test specification

In [120]:
datasets_path = config.DATA_BASE_PATH / 'test_datasets'
regex_datasets_name = 'test_dataset_*.tsv'
models_root = 'pairwise_wo_dupl_rf_out'
spec_name = 'rf_shuffled'

In [121]:
datasets_rf, instructions_rf = pairwise_test_yaml(datasets_path, regex_datasets_name, models_root, spec_name)

In [122]:
with open('shuffled_rf_test.yaml', 'w') as f:
        yaml.dump(
            {'definitions': {'datasets': datasets_rf},
            'instructions': instructions_rf},
            f,
            default_flow_style=False,
            sort_keys=False,
            Dumper=NoAliasDumper
        )

SVM test specification

In [93]:
models_root = 'pairwise_wo_dupl_svm_default_out'
spec_name = 'svm_shuffled'

In [94]:
datasets_svm, instructions_svm = pairwise_test_yaml(datasets_path, regex_datasets_name, models_root, spec_name)

In [96]:
with open('shuffled_svm_test_def.yaml', 'w') as f:
        yaml.dump(
            {'definitions': {'datasets': datasets_svm},
            'instructions': instructions_svm},
            f,
            default_flow_style=False,
            sort_keys=False,
            Dumper=NoAliasDumper
        )

KNN test specification

In [87]:
models_root = 'pairwise_wo_dupl_knn_default_out'
spec_name = 'knn_shuffled'

In [88]:
datasets_knn, instructions_knn = pairwise_test_yaml(datasets_path, regex_datasets_name, models_root, spec_name)

In [89]:
with open('shuffled_knn_test_def.yaml', 'w') as f:
        yaml.dump(
            {'definitions': {'datasets': datasets_knn},
            'instructions': instructions_knn},
            f,
            default_flow_style=False,
            sort_keys=False,
            Dumper=NoAliasDumper
        )

### Creating pairwise test datasets so that positive and negative datsets come from diffrent models.

In [127]:
antigens = ['3VRL', '1NSN', '3RAJ', '5E94', '1H0D', '1WEJ', '1ADQ', '1FBI','2YPV', '1OB1']

In [211]:
for i in antigens:
    !mkdir ../data/test_datasets/{i}

In [198]:
pos_neg_paits = dict()
dfs = []
for ag1 in antigens:
    paths = list(Path(datasets_path).glob(f'test_dataset_{ag1}_*.tsv'))
    pos_neg_paits[ag1] = []
    if len(paths):
        for path in paths:
            df_i = pd.read_csv(path, sep='\t', index_col=False)
            ag2 = re.search('test_dataset_.*_(.*).tsv', str(path)).group(1)
            pos_neg_paits[ag1].append(ag2)
            n = df_i.shape[0]
            df_i['positive_ag'] = [ag1]*n
            df_i['negative_ag'] = [ag2]*n
            dfs.append(df_i)
df_merged = pd.concat(dfs, axis=0)
        #df_merged.to_csv(f'../data/test_datasets/negatives/{ag1}_negatives.tsv', sep='\t')

In [212]:
for ag1, ag2s in pos_neg_paits.items():
    for ag2_excl in ag2s:
        print(ag1, ag2_excl)
        os.mkdir(f'../data/test_datasets/{ag1}/{ag1}_excl_{ag2_excl}')
        df_ag1 = df_merged[(df_merged['positive_ag'] == ag1) & (df_merged['negative_ag'] == ag2_excl)].copy(deep=True)
        df_ag1.drop(['Unnamed: 0', 'positive_ag', 'negative_ag'], axis=1, inplace=True)
        ag2s_copy =  set(ag2s)
        ag2_excl_set = ag2s_copy- set(ag2_excl)
        for ag2 in ag2_excl_set:
            df_ag2_excl = df_merged[(df_merged['positive_ag'] == ag1) & (df_merged['negative_ag'] == ag2)].copy(deep=True)
            df_ag2_excl.drop(['Unnamed: 0', 'positive_ag', 'negative_ag'], axis=1, inplace=True)
            df_ag1_ag2 = pd.concat([df_ag1, df_ag2_excl],axis=0)
            df_ag1_ag2.to_csv(f'../data/test_datasets/{ag1}/{ag1}_excl_{ag2_excl}/{ag1}_{ag2}.tsv',sep='\t');

3VRL 5E94
1NSN 1WEJ
1NSN 1OB1
1NSN 3RAJ
1NSN 2YPV
1NSN 3VRL
1NSN 5E94
3RAJ 3VRL
3RAJ 5E94
1H0D 1WEJ
1H0D 1OB1
1H0D 1NSN
1H0D 3VRL
1H0D 2YPV
1H0D 3RAJ
1H0D 5E94
1WEJ 5E94
1WEJ 3RAJ
1WEJ 2YPV
1WEJ 3VRL
1ADQ 1OB1
1ADQ 1NSN
1ADQ 1WEJ
1ADQ 1FBI
1ADQ 5E94
1ADQ 1H0D
1ADQ 3VRL
1ADQ 2YPV
1ADQ 3RAJ
1FBI 1WEJ
1FBI 1NSN
1FBI 1OB1
1FBI 3RAJ
1FBI 2YPV
1FBI 3VRL
1FBI 1H0D
1FBI 5E94
2YPV 5E94
2YPV 3RAJ
2YPV 3VRL
1OB1 1WEJ
1OB1 5E94
1OB1 2YPV
1OB1 3RAJ
1OB1 3VRL


In [157]:
df_merged.head()

,Unnamed: 0,UID,Slide,Antigen,binder,shuffled_label,positive_ag,negative_ag
0,0,5E94_5315811_03a,LGLLGWYFDVW,5E94,False,False,3VRL,5E94
1,1,3VRL_2739360_00a,CTKALLLHWYF,3VRL,True,False,3VRL,5E94
2,2,5E94_4355632_03a,HFMGYDVLWYF,5E94,False,False,3VRL,5E94
3,3,3VRL_267399_00a,CARTSFLLLPF,3VRL,True,True,3VRL,5E94
4,4,3VRL_31961_00a,CARLGLLWVHY,3VRL,True,True,3VRL,5E94


In [114]:
for ag1 in antigens:
    print(ag1)
    paths = list(Path(datasets_path).glob(f'test_dataset_{ag1}_*.tsv'))
    dfs = []
    if len(paths):
        for path in paths:
            df_i = pd.read_csv(path, sep='\t')
            df_i = df_i[df_i['Antigen']!=ag1]
            dfs.append(df_i)
        df_merged = pd.concat(dfs, axis=0)
        df_merged.to_csv(f'../data/test_datasets/negatives/{ag1}_negatives.tsv', sep='\t')

3VRL
1NSN
3RAJ
5E94
1H0D
1WEJ
1ADQ
1FBI
2YPV
1OB1


### Testing models on negative data they haven't seen. Creating yaml file for immuneML

In [122]:
def cross_test_yaml(models_path):
    datasets = dict()
    instructions = dict()
    config_paths = glob.glob(str(Path(config.IMMUNE_ML_BASE_PATH / models_path)) +'/fit_*') #config.IMMUNE_ML_BASE_PATH / 'pairwise_wo_dupl_rf_out'   fit_pairwise_dataset_ag1_ag2_instruction
    for config_path in config_paths:
        ag1, ag2_excl = re.search('fit_pairwise_dataset_(.*)_(.*)_instruction', config_path).groups()
        config_path = Path(config_path) / 'optimal_binder/zip/ml_settings_binder.zip'
        config_path = str(config_path)
        paths = glob.glob(str(Path(config.DATA_BASE_PATH / f'test_datasets/{ag1}/{ag1}_excl_{ag2_excl}')) + '/*.tsv')
        #config.DATA_BASE_PATH / 'test_datasets'
        for path in paths:
            ag2 = re.search(f"{ag1}_excl_{ag2_excl}/{ag1}_(.*).tsv", path).group(1)
            datasets[f'{ag1}_excl{ag2_excl}_negds_{ag2}'] = {
                    "format": "Generic",
                    "params": {
                        "path": path,
                        "is_repertoire": False,
                        "region_type": "FULL_SEQUENCE",
                         "column_mapping": {
                                "Slide": "sequence_aas",
                                 "UID": "sequence_identifiers"

                            },
                            "metadata_column_mapping": {
                                "Antigen": "Antigen",
                                "binder": "binder",
                            }}
            }
            instructions[f'{ag1}_excl{ag2_excl}_negds_{ag2}_instruction'] = {
                        "type": "MLApplication",
                        "dataset": f'{ag1}_excl{ag2_excl}_negds_{ag2}',
                        "config_path": config_path,
                        "number_of_processes": 4
            }
    return datasets, instructions

In [123]:
rf_cross_datasets, rf_cross_instructions = cross_test_yaml('pairwise_wo_dupl_rf_out')

In [127]:
svm_cross_datasets, svm_cross_instructions = cross_test_yaml('pairwise_wo_dupl_svm_default_out')

In [128]:
with open(Path(config.IMMUNE_ML_BASE_PATH / 'yaml_specifications/cross_test_svm.yaml'), 'w') as f:
        yaml.dump(
            {'definitions': {'datasets': svm_cross_datasets},
            'instructions': svm_cross_instructions},
            f,
            default_flow_style=False,
            sort_keys=False,
            Dumper=NoAliasDumper
        )

In [ ]:
## results of immuneMl in immuneML folder: shuffled_... , cross_...